# Station isochrone Generation

This notebook aims to try and generate isochrones of train stations

It will first attempt to merge isochrones of station entrances where available

Otherwise it will be a simple isochrone generation of the approximate centre of the station



In [7]:
#setting up supabase client
import os
import pandas as pd
from supabase import create_client, Client

#in the py webapp, keys will be stored in github secrets
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [3]:
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import os

import sys

# Get the path of the parent directory (the root of the project)
module_path = os.path.abspath(os.path.join('..'))

# Add the parent directory to sys.path
sys.path.append(module_path)

# Now you can import key.py, this is only done for testing purposes. actual live version should be stored in secret github
from key import ORS_KEY

#Personal api_key stored in a key.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key
ors_key: str = ORS_KEY


ors_client = client.Client(key=ors_key) 

# Querying station entrances from Supabase


The following blocks will load the various relevant tables from Supabase

In [14]:
import geopandas as gpd
from shapely.geometry import Point

#query to supabase using the previous supabase client that was declared
response_entrances_isochrones = supabase.table('entrances_isochrones').select("*").execute()

data,_ = response_entrances_isochrones
entrances_isochrones_df = pd.DataFrame(data[1])

# Convert the geometry column from dictionaries to shapely Polygons
entrances_isochrones_df['geometry'] = entrances_isochrones_df['geometry'].apply(lambda geom: shape(geom))

# Convert the DataFrame to a GeoDataFrame
entrances_isochrones_gdf = gpd.GeoDataFrame(entrances_isochrones_df, geometry='geometry', crs='EPSG:4326')


entrances_isochrones_gdf

,isochrone_id,value,center,area,reachfactor,total_pop,type,entrance_id,geometry
0,3308608988900,900,"[101.71272677986722, 3.158791494667739]",2964181.86,0.9435,42510,Polygon,3308608988,"POLYGON ((101.70289 3.15823, 101.70243 3.15634..."
1,3308608988300,300,"[101.71272677986722, 3.158791494667739]",356944.49,1.0000,5009,Polygon,3308608988,"POLYGON ((101.70929 3.15881, 101.70934 3.15874..."
2,3308608988600,600,"[101.71272677986722, 3.158791494667739]",1365017.65,0.9776,19193,Polygon,3308608988,"POLYGON ((101.70599 3.15737, 101.70598 3.15725..."
3,3308608989300,300,"[101.7125217751169, 3.1588555818871913]",337860.68,0.9679,4774,Polygon,3308608989,"POLYGON ((101.70906 3.15892, 101.70922 3.15865..."
4,3308608989600,600,"[101.7125217751169, 3.1588555818871913]",1362846.78,0.9761,18751,Polygon,3308608989,"POLYGON ((101.70580 3.15729, 101.70579 3.15715..."
...,...,...,...,...,...,...,...,...,...
877,12182196430600,600,"[101.72715278646551, 3.123366156533917]",1266713.44,0.9072,9056,Polygon,12182196430,"POLYGON ((101.72167 3.12239, 101.72191 3.12213..."
878,12182196430900,900,"[101.72715278646551, 3.123366156533917]",2972398.37,0.9461,23787,Polygon,12182196430,"POLYGON ((101.71862 3.12433, 101.71860 3.12223..."
879,12191934049300,300,"[101.73887678477966, 3.130344012142177]",106447.13,0.3049,1406,Polygon,12191934049,"POLYGON ((101.73711 3.13201, 101.73740 3.13131..."
880,12191934049600,600,"[101.73887678477966, 3.130344012142177]",463494.89,0.3320,5604,Polygon,12191934049,"POLYGON ((101.73501 3.13384, 101.73810 3.12974..."


In [6]:
#query to supabase using the previous supabase client that was declared
response_stations = supabase.table('stations').select("*").eq('region','Klang Valley').execute()

data,_ = response_stations

stations_df = pd.DataFrame(data[1])
# Create a new column in your DataFrame for the geographic data
stations_df['geometry'] = [Point(xy) for xy in zip(stations_df['longitude'], stations_df['latitude'])]

# Convert the DataFrame to a GeoDataFrame
stations_gdf = gpd.GeoDataFrame(stations_df, geometry='geometry')
# Set the coordinate reference system (CRS) to EPSG:4326 (WGS84)
stations_gdf.crs = "EPSG:4326"

stations_gdf

,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened,station_id,geometry
0,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,1,POINT (101.69379 3.13951)
1,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,2,POINT (101.69301 3.15454)
2,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,3,POINT (101.69123 3.16500)
3,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,4,POINT (101.67899 3.11853)
4,Seputeh,KB02,Keretapi Tanah Melayu,3.113697,101.681299,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,5,POINT (101.68130 3.11370)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Sri Andalas,JS22,Rapid KL,3.015225,101.440441,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,255,POINT (101.44044 3.01522)
255,Klang Jaya,JS23,Rapid KL,3.005072,101.442081,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,256,POINT (101.44208 3.00507)
256,Bandar Bukit Tinggi,JS24,Rapid KL,2.993526,101.446175,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,257,POINT (101.44617 2.99353)
257,Johan Setia,JS26,Rapid KL,2.975436,101.460718,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,258,POINT (101.46072 2.97544)


In [18]:
#query to supabase using the previous supabase client that was declared
response_stations = supabase.table('station_entrances').select("*").execute()

data,_ = response_stations

stations_entrances_df = pd.DataFrame(data[1])
# Create a new column in your DataFrame for the geographic data
stations_entrances_df

,relationship_id,entrance_id,station_name,station_code
0,0,10796851698,Pudu,AG10
1,1,10796851698,Pudu,SP10
2,2,5485710279,Kampung Baru,KJ11
3,3,5485710278,Kampung Baru,KJ11
4,4,9740843587,Masjid Jamek (KJ),KJ13
...,...,...,...,...
324,324,5044809585,Tun Razak Exchange (PY),PY23
325,325,5044809586,Tun Razak Exchange (PY),PY23
326,326,12155642296,Kajang 2,KB06A
327,327,12155642293,Kajang 2,KB06A


## Task TODO

1. Left join station_entrances with entrances_isochrones_gdf on entrance_id
2. This will create a table with isochrones mapped to station_name
3. Iterate stations table to look up joined table for entrance isochrones.
4. If isochrones are found, store all of them and use a unity function to combine them all
5. save in a new isochrones polygon column just for stations
6. If no isochrones found, just query on station coordinates

In [23]:
#correcting the column data type to prep for merging
entrances_isochrones_gdf['entrance_id']= entrances_isochrones_gdf['entrance_id'].astype(int)  

In [24]:
# Step 1: Left join station_entrances with entrances_isochrones_gdf on entrance_id
# This will create a table where each station entrance has corresponding isochrones mapped
merged_gdf = stations_entrances_df.merge(entrances_isochrones_gdf, on='entrance_id', how='left')
merged_gdf

,relationship_id,entrance_id,station_name,station_code,isochrone_id,value,center,area,reachfactor,total_pop,type,geometry
0,0,10796851698,Pudu,AG10,10796851698300,300.0,"[101.7121408, 3.1347568]",278905.63,0.7990,2478.0,Polygon,"POLYGON ((101.70965 3.13579, 101.70971 3.13562..."
1,0,10796851698,Pudu,AG10,10796851698600,600.0,"[101.7121408, 3.1347568]",1366234.08,0.9785,13013.0,Polygon,"POLYGON ((101.70670 3.13675, 101.70656 3.13563..."
2,0,10796851698,Pudu,AG10,10796851698900,900.0,"[101.7121408, 3.1347568]",2968558.89,0.9449,26537.0,Polygon,"POLYGON ((101.70388 3.13690, 101.70406 3.13641..."
3,1,10796851698,Pudu,SP10,10796851698300,300.0,"[101.7121408, 3.1347568]",278905.63,0.7990,2478.0,Polygon,"POLYGON ((101.70965 3.13579, 101.70971 3.13562..."
4,1,10796851698,Pudu,SP10,10796851698600,600.0,"[101.7121408, 3.1347568]",1366234.08,0.9785,13013.0,Polygon,"POLYGON ((101.70670 3.13675, 101.70656 3.13563..."
...,...,...,...,...,...,...,...,...,...,...,...,...
980,327,12155642293,Kajang 2,KB06A,12155642293600,600.0,"[101.7924757365604, 2.962197483109656]",1132110.23,0.8108,5671.0,Polygon,"POLYGON ((101.78562 2.96171, 101.78582 2.96056..."
981,327,12155642293,Kajang 2,KB06A,12155642293900,900.0,"[101.7924757365604, 2.962197483109656]",2647963.56,0.8429,13241.0,Polygon,"POLYGON ((101.78315 2.96410, 101.78342 2.96268..."
982,328,12155642294,Kajang 2,KB06A,12155642294300,300.0,"[101.7920585756707, 2.962773292823094]",275078.71,0.7880,1118.0,Polygon,"POLYGON ((101.78944 2.96378, 101.78966 2.96172..."
983,328,12155642294,Kajang 2,KB06A,12155642294600,600.0,"[101.7920585756707, 2.962773292823094]",1279931.08,0.9167,5476.0,Polygon,"POLYGON ((101.78577 2.96369, 101.78622 2.96092..."


In [25]:
stations_gdf


,name,station_code,service_provider_name,latitude,longitude,route_id,route_name,line_number,line_colour,colour_hex_code,region,odonym,namesake,opened,station_id,geometry
0,Kuala Lumpur,KA02,Keretapi Tanah Melayu,3.139513,101.693789,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,1,POINT (101.69379 3.13951)
1,Bank Negara,KA03,Keretapi Tanah Melayu,3.154542,101.693010,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,2,POINT (101.69301 3.15454)
2,Putra,KA04,Keretapi Tanah Melayu,3.165005,101.691234,KA,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,3,POINT (101.69123 3.16500)
3,Mid Valley,KB01,Keretapi Tanah Melayu,3.118528,101.678985,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,4,POINT (101.67899 3.11853)
4,Seputeh,KB02,Keretapi Tanah Melayu,3.113697,101.681299,KB,Seremban Line,1,Blue,#0000FF,Klang Valley,None,None,None,5,POINT (101.68130 3.11370)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Sri Andalas,JS22,Rapid KL,3.015225,101.440441,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,255,POINT (101.44044 3.01522)
255,Klang Jaya,JS23,Rapid KL,3.005072,101.442081,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,256,POINT (101.44208 3.00507)
256,Bandar Bukit Tinggi,JS24,Rapid KL,2.993526,101.446175,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,257,POINT (101.44617 2.99353)
257,Johan Setia,JS26,Rapid KL,2.975436,101.460718,JS,LRT3,11,Sky Blue,#88cffa,Klang Valley,None,None,None,258,POINT (101.46072 2.97544)


In [5]:
for idx, station_row in stations_gdf.iterrows():
    station_code= station_row['station_code']

    

list

In [6]:
coordinates= [entrances_gdf.iloc[50]['geometry'].coords[0][0],entrances_gdf.iloc[50]['geometry'].coords[0][1]]
entrance_id = entrances_gdf.iloc[50]['entrance_id']
type(coordinates)
#sample of parameters to be passed through to ors client and generate isochrones. in the form of dictionary
isochrone_params = {
              'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }

#add the specific key='locations' to be passed to the ors client. consists of a list of coordinates. coordinates in turn are to be in a list of [lat,long]
isochrone_params['locations'] = [coordinates]

#ors query results in a dict 
temp_iso = ors_client.isochrones(**isochrone_params)


In [7]:
import json

####
#the following seems redundant for now
#temp_iso_json = json.dumps(temp_iso)
#data = json.loads(temp_iso_json)
#df = pd.json_normalize(data['features'])
####

#transform query result into a dataframe
df = pd.json_normalize(temp_iso['features'])
df = df.drop(columns=['type','properties.group_index'])
df

,properties.value,properties.center,properties.area,properties.reachfactor,properties.total_pop,geometry.coordinates,geometry.type
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,"[[[101.695956, 3.158645], [101.697156, 3.15670...",Polygon
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,"[[[101.692989, 3.161272], [101.694041, 3.15613...",Polygon
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,"[[[101.692929, 3.163399], [101.692796, 3.16219...",Polygon


In [8]:
#simplify values in geometry.coordinates column. only first value in list is relevant
df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

# Remove 'properties.' and 'geometry.' from the column names
df.columns = df.columns.str.replace('properties.', '', regex=False)
df.columns = df.columns.str.replace('geometry.', '', regex=False)
df

,value,center,area,reachfactor,total_pop,coordinates,type
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,"[[101.695956, 3.158645], [101.697156, 3.156708...",Polygon
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,"[[101.692989, 3.161272], [101.694041, 3.156137...",Polygon
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,"[[101.692929, 3.163399], [101.692796, 3.16219]...",Polygon


In [9]:
#creating a new id key for isochrones. to be inserted in supabase table
df['entrance_id']=entrance_id
df['isochrone_id'] = df['entrance_id'].astype(str) + df['value'].astype(int).astype(str)
df

,value,center,area,reachfactor,total_pop,coordinates,type,entrance_id,isochrone_id
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,"[[101.695956, 3.158645], [101.697156, 3.156708...",Polygon,7259253706,7259253706300
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,"[[101.692989, 3.161272], [101.694041, 3.156137...",Polygon,7259253706,7259253706600
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,"[[101.692929, 3.163399], [101.692796, 3.16219]...",Polygon,7259253706,7259253706900


In [10]:
from shapely.geometry import Polygon

# Convert list of coordinates to Polygon objects
df['geometry'] = df['coordinates'].apply(Polygon)
df.drop(columns=['coordinates'],inplace=True)
# Convert to GeoDataFrame
entrances_iso_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")



In [11]:
entrances_iso_gdf

,value,center,area,reachfactor,total_pop,type,entrance_id,isochrone_id,geometry
0,300.0,"[101.6991072, 3.1590919]",285653.74,0.8183,2842.0,Polygon,7259253706,7259253706300,"POLYGON ((101.69596 3.15864, 101.69716 3.15671..."
1,600.0,"[101.6991072, 3.1590919]",1375757.45,0.9853,16064.0,Polygon,7259253706,7259253706600,"POLYGON ((101.69299 3.16127, 101.69404 3.15614..."
2,900.0,"[101.6991072, 3.1590919]",2935073.01,0.9343,42050.0,Polygon,7259253706,7259253706900,"POLYGON ((101.69293 3.16340, 101.69280 3.16219..."


In [12]:
#data preparation of geodataframe before being compatible for updating supabase
gpf_copy = entrances_iso_gdf.copy()
gpf_copy['geometry'] = gpf_copy['geometry'].apply(lambda geom: geom.wkt)
gpf_copy['area'] = gpf_copy['area'].astype(float)
gpf_copy['reachfactor'] = gpf_copy['reachfactor'].astype(float)
gpf_copy['total_pop'] = gpf_copy['total_pop'].astype(float)



/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [13]:
#upsert will insert the row if it doesn’t exist or update the row if it already exists. 
data, error = supabase.table('entrances_isochrones').upsert(gpf_copy.to_dict(('records'))).execute()



In [14]:
#creating a reusable function to generate isochrones
def generate_entrances_isochrone(client,entrance_id,reach_centre,range=900,interval=300):
    isochrone_parameters = {
              'profile': 'foot-walking', 
              'range': [range], # 900/60 = 15 mins
              'interval': interval,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }

    #reach_centre is in the format: [101.7127175, 3.1587619], ie [longitude,latitude]
    isochrone_parameters['locations'] = [reach_centre]
    isochrones_output = client.isochrones(**isochrone_parameters)

    df = pd.json_normalize(isochrones_output['features'])
    df = df.drop(columns=['type','properties.group_index'])

    #reformate coordinates column
    df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

    # Remove 'properties.' and 'geometry.' from the column names
    df.columns = df.columns.str.replace('properties.', '', regex=False)
    df.columns = df.columns.str.replace('geometry.', '', regex=False)
    

    #creating a new id key for isochrones. to be inserted in supabase table
    df['entrance_id']=entrance_id
    df['isochrone_id'] = df['entrance_id'].astype(str) + df['value'].astype(int).astype(str)
    df
    return df   
    

In [15]:
reach = [entrances_gdf.iloc[2]['geometry'].coords[0][0],entrances_gdf.iloc[2]['geometry'].coords[0][1]]
id = entrances_gdf.iloc[2]['entrance_id']
function_df = generate_entrances_isochrone(ors_client,id,reach)
function_df

,value,center,area,reachfactor,total_pop,coordinates,type,entrance_id,isochrone_id
0,300.0,"[101.6141298, 3.0222312]",220091.61,0.6305,1498.0,"[[101.611457, 3.023368], [101.611967, 3.021279...",Polygon,4092013971,4092013971300
1,600.0,"[101.6141298, 3.0222312]",958202.50,0.6863,5893.0,"[[101.608258, 3.021474], [101.608289, 3.021367...",Polygon,4092013971,4092013971600
2,900.0,"[101.6141298, 3.0222312]",2248676.02,0.7158,12267.0,"[[101.605448, 3.020465], [101.60581, 3.020405]...",Polygon,4092013971,4092013971900


In [16]:
def update__entrances_isochrone_supabase(supabase_client,isochrone_dataframe):
    # Convert list of coordinates to Polygon objects
    isochrone_dataframe['geometry'] = isochrone_dataframe['coordinates'].apply(Polygon)
    isochrone_dataframe.drop(columns=['coordinates'],inplace=True)
    
    # Convert to GeoDataFrame
    isochrone_geodataframe = gpd.GeoDataFrame(isochrone_dataframe, geometry='geometry', crs="EPSG:4326")
    isochrone_geodataframe['geometry'] = isochrone_geodataframe['geometry'].apply(lambda geom: geom.wkt)
    isochrone_geodataframe['area'] = isochrone_geodataframe['area'].astype(float)
    isochrone_geodataframe['reachfactor'] = isochrone_geodataframe['reachfactor'].astype(float)
    isochrone_geodataframe['total_pop'] = isochrone_geodataframe['total_pop'].astype(float)

    isochrone_dict = isochrone_geodataframe.to_dict(('records'))

    #upsert will insert the row if it doesn’t exist or update the row if it already exists. 
    data, error = supabase_client.table('entrances_isochrones').upsert(isochrone_dict).execute()

    return data,error


In [36]:
update__entrances_isochrone_supabase(supabase,function_df)

/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


(('data',
  [{'isochrone_id': '4092013971300',
    'value': 300,
    'center': '[101.6141298, 3.0222312]',
    'area': 220091.61,
    'reachfactor': 0.6305,
    'total_pop': 1498,
    'type': 'Polygon',
    'entrance_id': '4092013971',
    'geometry': {'type': 'Polygon',
     'coordinates': [[[101.611457, 3.023368],
       [101.611967, 3.021279],
       [101.612704, 3.020689],
       [101.613056, 3.020613],
       [101.614862, 3.021034],
       [101.616296, 3.022148],
       [101.61654, 3.022733],
       [101.616571, 3.022918],
       [101.616602, 3.023104],
       [101.616602, 3.023104],
       [101.616633, 3.023289],
       [101.616664, 3.023475],
       [101.615475, 3.025506],
       [101.615244, 3.025566],
       [101.613829, 3.025002],
       [101.612421, 3.02446],
       [101.611923, 3.024061],
       [101.611507, 3.023724],
       [101.611457, 3.023368]]]}},
   {'isochrone_id': '4092013971600',
    'value': 600,
    'center': '[101.6141298, 3.0222312]',
    'area': 958202.5,
   

In [19]:
def process_entrances(entrances_gdf, client, supabase_client):
    # Iterate over each row in the GeoDataFrame
    for index, row in entrances_gdf.iterrows():
        entrance_id = row['entrance_id']  # Extract entrance_id
        
        # Extract coordinates (reach_centre) from the geometry
        reach_centre = [row['geometry'].coords[0][0], row['geometry'].coords[0][1]]
        
        # Generate the isochrone for this entrance using the specified function
        try:
            isochrone_dataframe = generate_entrances_isochrone(
                client, 
                entrance_id, 
                reach_centre, 
                range=900,    # 15 minutes
                interval=300  # 5 minutes
            )
            
            # Update the isochrone data in Supabase using the specified function
            update__entrances_isochrone_supabase(supabase_client, isochrone_dataframe)
            print(f'updated {entrance_id}')
            
        except Exception as e:
            # Handle exceptions (e.g., API errors, invalid geometries, etc.)
            print(f"Error processing entrance {entrance_id}: {e}")


In [20]:
#setting up supabase client
import os
import pandas as pd
from supabase import create_client, Client

#in the py webapp, keys will be stored in github secrets
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)


import geopandas as gpd
from shapely.geometry import Point

#query to supabase using the previous supabase client that was declared
response = supabase.table('entrances').select("*").execute()
data,_ = response

entrances_df = pd.DataFrame(data[1])
# Create a new column in your DataFrame for the geographic data
entrances_df['geometry'] = [Point(xy) for xy in zip(entrances_df['longitude'], entrances_df['latitude'])]

# Convert the DataFrame to a GeoDataFrame
entrances_gdf = gpd.GeoDataFrame(entrances_df, geometry='geometry')
# Set the coordinate reference system (CRS) to EPSG:4326 (WGS84)
entrances_gdf.crs = "EPSG:4326"

#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import sys

# Get the path of the parent directory (the root of the project)
module_path = os.path.abspath(os.path.join('..'))

# Add the parent directory to sys.path
sys.path.append(module_path)

# Now you can import key.py, this is only done for testing purposes. actual live version should be stored in secret github
from key import ORS_KEY

#Personal api_key stored in a key.py file that is in gitignore. Uncomment the following and provide your own ORS api for your own use
## api_key = "you api key " #Provide your personal API key
ors_key: str = ORS_KEY
ors_client = client.Client(key=ors_key) 

process_entrances(entrances_gdf, ors_client, supabase)

/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 3308608988


/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 3308608989


/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


...

/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 5042521364


/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


updated 12182196430
updated 12191934049


/home/nabilersyad/miniconda3/envs/sql/lib/python3.9/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [21]:
#creating a reusable function to generate isochrones
#TODO allow batch processing by accepting dataframes and do a batch api call to generate isochrones
def generate_entrances_isochrone(client,entrance_id,reach_centre,range=900,interval=300):
    isochrone_parameters = {
              'profile': 'foot-walking', 
              'range': [range], # 900/60 = 15 mins
              'interval': interval,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }

    #reach_centre is in the format: [101.7127175, 3.1587619], ie [longitude,latitude]
    isochrone_parameters['locations'] = [reach_centre]
    isochrones_output = client.isochrones(**isochrone_parameters)

    df = pd.json_normalize(isochrones_output['features'])
    df = df.drop(columns=['type','properties.group_index'])

    #reformate coordinates column
    df['geometry.coordinates']=df['geometry.coordinates'].apply(lambda row:row[0])

    # Remove 'properties.' and 'geometry.' from the column names
    df.columns = df.columns.str.replace('properties.', '', regex=False)
    df.columns = df.columns.str.replace('geometry.', '', regex=False)
    

    #creating a new id key for isochrones. to be inserted in supabase table
    df['entrance_id']=entrance_id
    df['isochrone_id'] = df['entrance_id'].astype(str) + df['value'].astype(int).astype(str)
    df
    return df   